In [1]:
push!(LOAD_PATH, "/home/amir/Work/gdmrg")
import Tmp

In [15]:
reload("Tmp")

In [3]:
Tmp.runstuff(8,100)

8×8 Array{Float64,2}:
  0.5          -0.431043     -7.77156e-16  …  -6.80012e-16   0.114677   
 -0.431043      0.5          -0.247409        -0.0169671     1.38778e-17
 -7.77156e-16  -0.247409      0.5              6.93889e-16  -0.131644   
  0.183634      7.63278e-17  -0.379053         0.0519901    -2.22045e-16
  2.22045e-16   0.0519901    -2.63678e-16     -1.20737e-15   0.183634   
 -0.131644     -1.11022e-16   0.166667     …  -0.247409      4.44089e-16
 -6.80012e-16  -0.0169671     6.93889e-16      0.5          -0.431043   
  0.114677      1.38778e-17  -0.131644        -0.431043      0.5        

8×8 Array{Float64,2}:
  1.0           3.4521e-16    4.48426e-16  …   1.70855e-15   1.50747e-15
  2.25297e-16   0.334277     -0.379837        -1.76686e-15   0.132323   
  3.75568e-16  -0.379837      0.460943        -0.0268826    -0.0811366  
  1.64799e-16   0.189149     -0.292083         0.0464098    -0.0883982  
 -1.38778e-17   0.0435749     0.0538938        0.0106916     0.179709   
  1.12063e-15  -0.151901      0.0931412    …  -0.247409      4.44089e-16
  1.70855e-15  -1.76686e-15  -0.0268826        0.5          -0.431043   
  1.50747e-15   0.132323     -0.0811366       -0.431043      0.5        

8×8 Array{Float64,2}:
  1.0           5.87398e-16  -8.6059e-18   …   1.70855e-15   1.50747e-15
  4.60422e-16  -4.50816e-18   1.82465e-16      5.29091e-17  -1.17961e-16
  4.86448e-17   2.06432e-16   0.5              2.1992e-15   -0.161834   
 -4.7475e-17   -1.249e-16    -0.435082         0.0546887     1.04916e-14
 -5.05199e-17   6.93889e-18   9.09342e-15     -3.18148e-15   0.194967   
  1.12063e-15   6.93889e-17   0.185778     …  -0.247409      4.44089e-16
  1.70855e-15   5.29091e-17   2.1992e-15       0.5          -0.431043   
  1.50747e-15  -1.17961e-16  -0.161834        -0.431043      0.5        

8×8 Array{Float64,2}:
  1.0           5.87398e-16  -1.48443e-16  …   1.70855e-15   1.50747e-15
  4.60422e-16  -4.50816e-18   2.16126e-16      5.29091e-17  -1.17961e-16
 -1.346e-16     1.91097e-16   1.0              3.46945e-17   7.10015e-17
  1.45262e-16   1.22115e-16  -1.249e-16        2.59515e-15  -0.192882   
 -3.61589e-16  -8.38364e-17  -1.52656e-16      0.0962528     0.151995   
  1.04548e-15   6.79583e-17   2.08167e-17  …  -0.234387      0.0624178  
  1.70855e-15   5.29091e-17   3.46945e-17      0.5          -0.431043   
  1.50747e-15  -1.17961e-16   7.10015e-17     -0.431043      0.5        

8×8 Array{Float64,2}:
  1.0           5.87398e-16  -1.48443e-16  …   1.70855e-15   1.50747e-15
  4.60422e-16  -4.50816e-18   2.16126e-16      5.29091e-17  -1.17961e-16
 -1.346e-16     1.91097e-16   1.0              3.46945e-17   7.10015e-17
  1.41902e-16   3.98055e-17  -1.82471e-16      0.0          -3.46945e-17
 -6.99422e-17  -1.26508e-16   8.63311e-18     -3.41394e-15   0.253381   
  1.10446e-15   9.47113e-17   7.7246e-17   …  -0.253381     -1.38084e-15
  1.70855e-15   5.29091e-17   3.46945e-17      0.5          -0.431043   
  1.50747e-15  -1.17961e-16   7.10015e-17     -0.431043      0.5        

...

8×8 Array{Float64,2}:
  1.0           5.87398e-16  -1.48443e-16  …   2.11841e-15   1.34792e-17
  4.60422e-16  -4.50816e-18   2.16126e-16     -1.5626e-17   -1.51196e-16
 -1.346e-16     1.91097e-16   1.0              3.76206e-17   6.27907e-17
  1.41902e-16   3.98055e-17  -1.82471e-16     -4.42629e-17  -4.80253e-18
  1.35057e-15   8.81256e-18   7.23161e-17      1.7541e-17    8.4601e-16 
  2.11375e-16   1.25129e-16   3.69263e-17  …  -1.37383e-16   1.96262e-17
  2.13252e-15  -5.82619e-18   6.82614e-17     -2.2803e-31   -7.85046e-17
 -6.28957e-19  -1.60996e-16   3.21499e-17     -1.11022e-16   1.0        

8-element Array{Float64,1}:
  1.0        
 -4.50816e-18
  1.0        
 -6.37689e-17
 -2.03007e-17
  1.0        
 -2.2803e-31 
  1.0        

8-element Array{Int64,1}:
 1
 0
 1
 0
 0
 1
 0
 1

making correlation matrix...
generating the gates(GMPS)...
forming MPS from the GMPS gates...


LoadError: [91mMethodError: no method matching Tmp.MPS(::Int64, ::Int64, ::Array{Int64,1})[0m
Closest candidates are:
  Tmp.MPS(::Any, ::Any, ::Any, [91m::Any[39m, [91m::Any[39m) at /home/amir/Work/gdmrg/MPS.jl:5[39m

In [16]:
singlet = Vector{Complex128}([0, 1, -1, 0])
ketstate = kron(singlet, singlet)
Tmp.MPS(4, 2^4, 2, ketstate)

S = [1.41421, 1.41421]
ratio = 1.0


1.0